In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dt = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
dt.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dt.shape

(42000, 785)

In [5]:
data = np.array(dt)
m, n = data.shape
np.random.shuffle(data)

data_test = data[0:1000].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test / 255  

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255 # NORMALIZE for better trai

In [6]:
X_train[:,0].shape

(784,)

In [7]:
def init_param():
    W1 = np.random.randn(128, 784) * 0.01 # scaling weights approprately & increase size
    b1 = np.zeros((128,1))

    W2 = np.random.randn(64, 128) * 0.01 # adding second layer (64 neurons)
    b2 = np.zeros((64,1))

    W3 = np.random.randn(10, 64) * 0.01 # output
    b3 = np.zeros((10,1))

    return W1, b1, W2, b2, W3, b3

def ReLu(X):
    return np.maximum(0, X)

def softmax(Z):
    #STABLE SOFTMAX
    Z_shifted = Z - np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z_shifted)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLu(Z1)

    Z2 = W2.dot(A1) +b2
    A2 = softmax(Z2)

    Z3 = W3.dot(A2) +b3
    A3 = softmax(Z3)

    return Z1, A1, Z2, A2, Z3, A3


def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, 10))  #blank canvas (10, 10)
    one_hot_Y[np.arange(Y.size), Y] = 1
    #one_hot_Y[0, Y[0]] = 1
    #one_hot_Y[1, Y[1]] = 1
    #...
    #one_hot_Y[Y.size -1, Y[size-1]] = 1

    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def drv_relu(X):
    return (X > 0).astype(float)

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)

    dZ3 = A3 - one_hot_Y
    dW3 = 1/m * dZ3.dot(A2.T)
    db3 = 1/m * np.sum(dZ3, axis=1, keepdims=True)
    
    dZ2 = W3.T.dot(dZ3) * drv_relu(Z2)
    dW2 = 1/m * dZ2.dot(A1.T) 
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)

    
    dZ1 = W2.T.dot(dZ2) * drv_relu(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3

def update_param(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, a):
    W1 = W1- a * dW1
    b1 = b1 - a * db1
    
    W2 = W2 - a * dW2
    b2 = b2 - a * db2

    W3 = W3 - a * dW3
    b3 = b3 - a * db3
    
    return W1, b1, W2, b2, W3, b3    
    
    
    
    

In [8]:
def get_predictions(A3):
    return np.argmax(A3, 0) #the index of the highest probability across columns.

def get_accuracy(prediction, Y):
    if len(prediction) > 10:
        print("Predictions:", prediction[:10], "True:", Y[:10])
    else:
        print("Predictions:", prediction, "True:", Y)
    return np.sum(prediction == Y) / Y.size
    return np.sum(prediction == Y) / Y.size # (Number of Correct) / (Total Number)

In [9]:
def gradiant_decent(X, Y, iterations, a):
    W1, b1, W2, b2, W3, b3 = init_param()

    for i in range(iterations):
        # Learing rate decay
        learning_rate = a * (0.95 ** (i//50)) # reduce 5% every 50 itrs
        
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_param(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate)

        if (i % 10 == 0):
            print(f'Iteration: {i}, Learning rate: {learning_rate:.6f}')
            print('accuracy : ', get_accuracy(get_predictions(A3), Y))
    return W1, b1, W2, b2, W3, b3

In [10]:
W1, b1, W2, b2, W3, b3 = gradiant_decent(X_train, Y_train, 200, 0.001)

Iteration: 0, Learning rate: 0.001000
Predictions: [4 4 4 4 4 4 4 4 4 4] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.0966829268292683
Iteration: 10, Learning rate: 0.001000
Predictions: [4 4 4 4 4 4 4 4 4 4] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.0966829268292683
Iteration: 20, Learning rate: 0.001000
Predictions: [4 4 4 4 4 4 4 4 4 4] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.0966829268292683
Iteration: 30, Learning rate: 0.001000
Predictions: [4 4 4 4 4 4 4 4 4 4] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.0966829268292683
Iteration: 40, Learning rate: 0.001000
Predictions: [7 7 7 7 7 7 7 7 7 7] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.10482926829268292
Iteration: 50, Learning rate: 0.000950
Predictions: [7 7 7 7 7 7 7 7 7 7] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.10482926829268292
Iteration: 60, Learning rate: 0.000950
Predictions: [7 7 7 7 7 7 7 7 7 7] True: [1 5 8 3 0 0 1 7 8 2]
accuracy :  0.10482926829268292
Iteration: 70, Learning rate: 0.000950
Predictions: [7 7 7 7 7 7 7 